<a href="https://colab.research.google.com/github/kokojay91/AIFFEL_quest_cr/blob/main/MainQuest%20/Quest02/MAIN_QUEST_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pokemon with stats
---

 포켓몬 게임 데이터를 기반으로 한 데이터 분석 결과와 모델 학습 결과를 설명한다.
  Decision Tree 모델과 그 외 비교 모델들을 사용하여 포켓몬이 전설 포켓몬인지 일반 포켓몬인지 예측하는 모델을 만드는 것이 목표이다.



## STEP 0. 목차
---

1. 데이터 살펴보기
    - 데이터를 읽어들여 온다.
    - https://www.kaggle.com/ 사이트를 참고하여, feature들의 의미를 확인한다.
    - 결측치 존재 여부를 확인한다.
2. 기본 모델이 동작하는 코드 제출하기
    - Feature Engineering (데이터 전처리)을 수행하기 전의 상태에서 기본적인 Decision Tree 모델을 학습
    -  데이터 전처리 없이 진행하라는 요구사항은 문자열 데이터는 제외하고 수치형 데이터만을 사용하는 방식으로 해결
3. 데이터 전처리
    - **데이터 살펴보기**에서 확인한 이상치에 대한 처리를 진행한다.
    - 불필요한 feature가 있다면 제거하고, 추가적으로 필요 feature가 있다면 생성하여 데이터에 추가 시켜준다.
4. 베이스라인 모델 학습 성공 및 분석
    - Decision Tree 모델을 학습하고 진행한 후 결과를 확인한다.
5. 비교 모델 학습 성공 및 비교 분석
    - Decision Tree 이외의 모델을 학습, 진행후 결과를 확인한다.
5. 제출
    - Padlet 게시판의 메인 퀘스트 제출 폼에 Notebook 파일을 업로드한다.


## STEP 1. 데이터 살펴보기
---

### Load Data
https://www.kaggle.com/ 사이트에서 pokemon with stats 데이터를 다운로드 후 읽어들여 온다.

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abcsds/pokemon")

print("Path to dataset files:", path)


100%|██████████| 14.9k/14.9k [00:00<00:00, 9.65MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/abcsds/pokemon/versions/2


In [6]:
import os

# 다운로드된 경로 확인
dataset_path = "/root/.cache/kagglehub/datasets/abcsds/pokemon/versions/2"

# 해당 경로에 파일이 있는지 확인
os.listdir(dataset_path)


['Pokemon.csv']

In [36]:
# CSV 파일 경로 결합
csv_path = dataset_path + "/Pokemon.csv"

# CSV 파일을 읽어 original_data 변수에 저장
original_data = pd.read_csv(csv_path)

# DataFrame 복사
pokemon = original_data.copy()

print(pokemon.shape)
pokemon.head()

(800, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [32]:
# 전설의 포켓몬 데이터셋
legendary = pokemon[pokemon["Legendary"] == True].reset_index(drop=True)
print(legendary.shape)
legendary.head()

(65, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,144,Articuno,Ice,Flying,580,90,85,100,95,125,85,1,True
1,145,Zapdos,Electric,Flying,580,90,90,85,125,90,100,1,True
2,146,Moltres,Fire,Flying,580,90,100,90,125,85,90,1,True
3,150,Mewtwo,Psychic,NaN,680,106,110,90,154,90,130,1,True
4,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True


In [31]:
# Q. 일반 포켓몬의 데이터셋도 만들어봅시다.
ordinary = pokemon[pokemon["Legendary"] == False].reset_index(drop=True)
print(ordinary.shape)
ordinary.head()

(735, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [35]:
# 전체 컬럼 출력하기
print(len(pokemon.columns))
pokemon.columns

13


Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

### Date Description
https://www.kaggle.com/ 페이지에서 확인할 수 있었던 데이터의 정보는 다음과 같다.

### Data fields
- '#' : 각 포켓몬의 ID
- 이름 : 각 포켓몬의 이름
- 유형 1 : 각 포켓몬은 유형을 가지고 있으며, 이는 공격에 대한 약점/저항력을 - 결정합니다.
- 2형 : 일부 포켓몬은 듀얼타입이며 2가지를 가지고 있습니다.
- 총합 : 이후의 모든 통계의 합계, 포켓몬의 강도에 대한 일반적인 가이드
- HP : 체력, 즉 생명력은 포켓몬이 기절하기 전에 얼마나 많은 피해를 견딜 수 있는지를 정의합니다.
- 공격 : 일반 공격(예: 스크래치, 펀치)에 대한 기본 수정치
- 방어력 : 일반 공격에 대한 기본 데미지 저항력
- SP Atk : 특수 공격, 특수 공격의 기본 수정치(예: 화염 폭발, 버블 빔)
- SP 방어력 : 특수 공격에 대한 기본 데미지 저항력
- 속도 : 각 라운드마다 어떤 포켓몬이 먼저 공격하는지 결정합니다.

### 결측치 확인

In [33]:
pokemon.isnull().sum()

,0
#,0
Name,0
Type 1,0
Type 2,386
Total,0
HP,0
Attack,0
Defense,0
Sp. Atk,0
Sp. Def,0


## STEP 2. 기본 모델이 동작하는 코드 제출하기
---

In [37]:
# 데이터불러오기

print(original_data.shape)
original_data.head()

(800, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [ ]:
# 라이브러리 불러우기

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [60]:
# 타겟 변수
target = 'Legendary'

# feature 변수 (타겟을 제외한 나머지 컬럼들)
# 문자열 컬럼인 'Name', 'Type 1', 'Type 2'는 제외, 숫자형 데이터만 남기기
features = pokemon.drop(columns=[target, 'Name', 'Type 1', 'Type 2'])

# 훈련 데이터와 테스트 데이터로 나누가 (80% 훈련, 20% 테스트)
X = features
y = pokemon[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# DecisionTreeClassifier 모델 학습
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# 모델 예측 (test 데이터로 예측)
y_pred = model.predict(X_test)

#  예측 결과 출력
print("예측 결과:", y_pred)

예측 결과: [ True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False  True False
 False False False False False False False False False False False False
 False False False False False  True False False False False  True False
 False False False False False False False False  True False False False
 False False False False False False False  True False False False False
  True False False False False False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False  True False False
  True False False False]


In [58]:
# 모델 성능 평가
cm = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[147   3]
 [  3   7]]


In [59]:
# 모델 성능 평가
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

       False       0.98      0.98      0.98       150
        True       0.70      0.70      0.70        10

    accuracy                           0.96       160
   macro avg       0.84      0.84      0.84       160
weighted avg       0.96      0.96      0.96       160



## STEP 3. 데이터 전처리
---

In [61]:
# 띄어쓰기가 없는 컬럼을 따로 만들어준 후, 띄어쓰기를 빈칸으로 처리
pokemon["Name_nospace"] = pokemon["Name"].apply(lambda i: i.replace(" ", ""))
pokemon.tail()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Name_nospace
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True,Diancie
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True,DiancieMegaDiancie
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True,HoopaHoopaConfined
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True,HoopaHoopaUnbound
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True,Volcanion


In [65]:
#  알파벳이 아닌 다른 문자가 이름에 포함된 것 확인하기
print(pokemon[pokemon["name_isalpha"] == False].shape)
pokemon[pokemon["name_isalpha"] == False]

(9, 15)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Name_nospace,name_isalpha
34,29,Nidoran♀,Poison,NaN,275,55,47,52,40,40,41,1,False,Nidoran♀,False
37,32,Nidoran♂,Poison,NaN,273,46,57,40,40,40,50,1,False,Nidoran♂,False
90,83,Farfetch'd,Normal,Flying,352,52,65,55,58,62,60,1,False,Farfetch'd,False
131,122,Mr. Mime,Psychic,Fairy,460,40,45,65,100,120,90,1,False,Mr.Mime,False
252,233,Porygon2,Normal,NaN,515,85,80,90,105,95,60,2,False,Porygon2,False
270,250,Ho-oh,Fire,Flying,680,106,130,90,110,154,90,2,True,Ho-oh,False
487,439,Mime Jr.,Psychic,Fairy,310,20,25,45,70,90,60,4,False,MimeJr.,False
525,474,Porygon-Z,Normal,NaN,535,85,80,70,135,75,90,4,False,Porygon-Z,False
794,718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95,6,True,Zygarde50%Forme,False


In [66]:
# 특정 행들의 'Name' 컬럼 값 확인
pokemon.loc[[34, 37, 90, 131, 252, 270, 487, 525, 794], 'Name']

,Name
34,Nidoran♀
37,Nidoran♂
90,Farfetch'd
131,Mr. Mime
252,Porygon2
270,Ho-oh
487,Mime Jr.
525,Porygon-Z
794,Zygarde50% Forme


In [70]:
# 문자열을 원하는 다른 문자열로 바꾸기
pokemon = pokemon.replace(to_replace="Nidoran♀", value="Nidoran X")
pokemon = pokemon.replace(to_replace="Nidoran♂", value="Nidoran Y")
pokemon = pokemon.replace(to_replace="Farfetch'd", value="Farfetchd")
pokemon = pokemon.replace(to_replace="Mr. Mime", value="Mr Mime")
pokemon = pokemon.replace(to_replace="Porygon2", value="Porygon Two")
pokemon = pokemon.replace(to_replace="Ho-oh", value="Ho Oh")
pokemon = pokemon.replace(to_replace="Mime Jr.", value="Mime Jr")
pokemon = pokemon.replace(to_replace="Porygon-Z", value="Porygon Z")
pokemon = pokemon.replace(to_replace="Zygarde50%Forme", value="Zygarde Forme")

pokemon.loc[[34, 37, 90, 131, 252, 270, 487, 525, 794]]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Name_nospace,name_isalpha
34,29,Nidoran X,Poison,NaN,275,55,47,52,40,40,41,1,False,Nidoran X,False
37,32,Nidoran Y,Poison,NaN,273,46,57,40,40,40,50,1,False,Nidoran Y,False
90,83,Farfetchd,Normal,Flying,352,52,65,55,58,62,60,1,False,Farfetchd,False
131,122,Mr Mime,Psychic,Fairy,460,40,45,65,100,120,90,1,False,Mr.Mime,False
252,233,Porygon Two,Normal,NaN,515,85,80,90,105,95,60,2,False,Porygon Two,False
270,250,Ho Oh,Fire,Flying,680,106,130,90,110,154,90,2,True,Ho Oh,False
487,439,Mime Jr,Psychic,Fairy,310,20,25,45,70,90,60,4,False,MimeJr.,False
525,474,Porygon Z,Normal,NaN,535,85,80,70,135,75,90,4,False,Porygon Z,False
794,718,Zygarde Forme,Dragon,Ground,600,108,100,121,81,95,95,6,True,Zygarde Forme,False


In [72]:
# 바꿔준 'Name' 컬럼으로 'Name_nospace'를 만들고, 다시 isalpha()로 체크해봅시다.
pokemon["Name_nospace"] = pokemon["Name"].apply(lambda i: i.replace(" ", ""))
pokemon["name_isalpha"] = pokemon["Name_nospace"].apply(lambda x: x.isalpha())

pokemon[pokemon["name_isalpha"] == False]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Name_nospace,name_isalpha


# 토근화 하기

In [93]:
# 토큰화 과정을 모든 이름에 적용
def tokenize(name):
    tokens = []
    a = re.findall('[A-Z][a-z]*', name)
    tokens.extend(a)

    return np.array(tokens)

pokemon['Name_tokens'] = pokemon['Name'].apply(tokenize)

In [95]:
# 전설의 포켓몬 데이터셋에 적용시켜서, 전설의 포켓몬의 이름에 사용된 토큰 모아보
all_tokens = list(legendary["Name"].apply(tokenize).values)

token_set = []
for token in all_tokens:
    token_set.extend(token)

print(len(set(token_set)))
print(token_set)

65
['Articuno', 'Zapdos', 'Moltres', 'Mewtwo', 'Mewtwo', 'Mega', 'Mewtwo', 'X', 'Mewtwo', 'Mega', 'Mewtwo', 'Y', 'Raikou', 'Entei', 'Suicune', 'Lugia', 'Ho', 'Regirock', 'Regice', 'Registeel', 'Latias', 'Latias', 'Mega', 'Latias', 'Latios', 'Latios', 'Mega', 'Latios', 'Kyogre', 'Kyogre', 'Primal', 'Kyogre', 'Groudon', 'Groudon', 'Primal', 'Groudon', 'Rayquaza', 'Rayquaza', 'Mega', 'Rayquaza', 'Jirachi', 'Deoxys', 'Normal', 'Forme', 'Deoxys', 'Attack', 'Forme', 'Deoxys', 'Defense', 'Forme', 'Deoxys', 'Speed', 'Forme', 'Uxie', 'Mesprit', 'Azelf', 'Dialga', 'Palkia', 'Heatran', 'Regigigas', 'Giratina', 'Altered', 'Forme', 'Giratina', 'Origin', 'Forme', 'Darkrai', 'Shaymin', 'Land', 'Forme', 'Shaymin', 'Sky', 'Forme', 'Arceus', 'Victini', 'Cobalion', 'Terrakion', 'Virizion', 'Tornadus', 'Incarnate', 'Forme', 'Tornadus', 'Therian', 'Forme', 'Thundurus', 'Incarnate', 'Forme', 'Thundurus', 'Therian', 'Forme', 'Reshiram', 'Zekrom', 'Landorus', 'Incarnate', 'Forme', 'Landorus', 'Therian', 'Form

In [97]:
# token_set에는 어떤 토큰이 가장 많이 쓰였는지 찾아보
from collections import Counter

most_common = Counter(token_set).most_common(10)
most_common

[('Forme', 15),
 ('Mega', 6),
 ('Mewtwo', 5),
 ('Kyurem', 5),
 ('Deoxys', 4),
 ('Hoopa', 4),
 ('Latias', 3),
 ('Latios', 3),
 ('Kyogre', 3),
 ('Groudon', 3)]

In [98]:
# 전설의 포켓몬 이름에 등장하는 토큰이 포켓몬의 이름에 있는지의 여부를 나타내는 컬럼을 만들기
for token, _ in most_common:
    # pokemon[token] = ... 형식으로 사용하면 뒤에서 warning이 발생합니다
    pokemon[f"{token}"] = pokemon["Name"].str.contains(token)

pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,...,Forme,Mega,Mewtwo,Kyurem,Deoxys,Hoopa,Latias,Latios,Kyogre,Groudon
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,...,False,False,False,False,False,False,False,False,False,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,...,False,False,False,False,False,False,False,False,False,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,...,False,False,False,False,False,False,False,False,False,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,...,False,True,False,False,False,False,False,False,False,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,...,False,False,False,False,False,False,False,False,False,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,...,False,False,False,False,False,False,False,False,False,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,...,False,False,False,False,False,False,False,False,False,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,...,False,True,False,False,False,False,False,False,False,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,...,False,True,False,False,False,False,False,False,False,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,...,False,False,False,False,False,False,False,False,False,False


# Type1 & 2! 범주형 데이터 전처리하기

In [102]:
# 각 속성의 종류는 총 몇 가지인지 알아보
len(list(set(pokemon["Type 1"]))), len(list(set(pokemon["Type 2"])))

(18, 19)

In [103]:
# 차집합 알아보기
set(pokemon["Type 2"]) - set(pokemon["Type 1"])

{nan}

In [104]:
# 포켓몬들의 모든 Type을 types 변수에 저장하기
types = list(set(pokemon["Type 1"]))
print(len(types))
print(types)

18
['Flying', 'Rock', 'Electric', 'Fighting', 'Grass', 'Bug', 'Normal', 'Fire', 'Ice', 'Dark', 'Steel', 'Fairy', 'Poison', 'Psychic', 'Ghost', 'Dragon', 'Water', 'Ground']


In [105]:
# 데이터가 비어있는 NaN값의 개수를 확인
pokemon["Type 2"].isna().sum()

386

In [107]:
# 18개의 컬럼에 대해 원-핫 인코딩을 해주기
for t in types:
    pokemon[t] = (pokemon["Type 1"] == t) | (pokemon["Type 2"] == t)

pokemon[[["Type 1", "Type 2"] + types][0]].head()

,Type 1,Type 2,Flying,Rock,Electric,Fighting,Grass,Bug,Normal,Fire,Ice,Dark,Steel,Fairy,Poison,Psychic,Ghost,Dragon,Water,Ground
0,Grass,Poison,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False
1,Grass,Poison,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False
2,Grass,Poison,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False
3,Grass,Poison,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False
4,Fire,NaN,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


## STEP 4. 베이스라인 모델 학습 성공 및 분석
---

In [181]:
model = DecisionTreeClassifier(random_state=25)
model

DecisionTreeClassifier(random_state=25)

In [182]:
# 모델 학습에 사용할 컬럼들만 추려서 features라는 변수에 저장
features = ['Total', 'HP', 'Attack', 'Defense','Sp. Atk', 'Sp. Def', 'Speed', 'Generation',
             'Forme', 'Mega', 'Mewtwo','Deoxys', 'Kyurem', 'Latias', 'Latios',
            'Kyogre', 'Groudon', 'Hoopa','Poison', 'Ground', 'Flying', 'Normal', 'Water', 'Fire',
            'Electric','Rock', 'Dark', 'Fairy', 'Steel', 'Ghost', 'Psychic', 'Ice', 'Bug', 'Grass', 'Dragon', 'Fighting']

len(features)

36

In [183]:
target = "Legendary"
target

'Legendary'

In [184]:
# 사용할 feature에 해당하는 데이터를 'X' 변수에 저장
X = pokemon[features]

# 'Legendary' 컬럼을 'y'에 저장
y = pokemon['Legendary']

In [185]:
# 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

# test 데이터로 모델의 예측 값을 얻기
model.fit(X_train, y_train)

# 모델 예측 (test 데이터로 예측)
y_pred = model.predict(X_test)

# 예측 결과 출력
print("예측 결과:", y_pred)

예측 결과: [ True False  True False False False False False False False False False
 False False False False False  True False False  True False False False
 False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
  True False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False  True False
 False False False  True False False False False False False False False
 False False False False False False False False  True False False False
  True False False  True False False  True False False False False False
 False False False False False False False  True False False False False
  True False False False False  True False False False False False False
  True False False False]


In [198]:
cm = confusion_matrix(y_test, y_pred)

# Confusion Matrix 출력
print("Confusion Matrix:")
print(cm)

# Classification Report 출력
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Confusion Matrix:
[[135  15]
 [  8   2]]
Classification Report:
              precision    recall  f1-score   support

       False       0.94      0.90      0.92       150
        True       0.12      0.20      0.15        10

    accuracy                           0.86       160
   macro avg       0.53      0.55      0.53       160
weighted avg       0.89      0.86      0.87       160



## STEP 5. 비교 모델 학습 성공 및 비교분
---

In [196]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

# 모델을 위한 feature와 target 설정
target = "Legendary"

# 사용할 feature 리스트
features = ['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation',
            'Forme', 'Mega', 'Mewtwo', 'Deoxys', 'Kyurem', 'Latias', 'Latios',
            'Kyogre', 'Groudon', 'Hoopa', 'Poison', 'Ground', 'Flying', 'Normal', 'Water', 'Fire',
            'Electric', 'Rock', 'Dark', 'Fairy', 'Steel', 'Ghost', 'Psychic', 'Ice', 'Bug', 'Grass', 'Dragon', 'Fighting']

# X와 y 변수 준비
X = pokemon[features]  # feature 데이터
y = pokemon[target]    # target 변수

# 훈련 데이터와 테스트 데이터로 분리 (80% 훈련, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **데이터 스케일링** - Logistic Regression은 스케일링이 중요함
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # 훈련 데이터 스케일링
X_test_scaled = scaler.transform(X_test)       # 테스트 데이터 스케일링 (훈련 데이터의 스케일러 사용)

# Logistic Regression 모델 초기화
model_lr = LogisticRegression(random_state=42)

# 모델 학습
model_lr.fit(X_train_scaled, y_train)

# 모델 예측
y_pred_lr = model_lr.predict(X_test_scaled)

# Confusion Matrix 출력
cm_lr = confusion_matrix(y_test, y_pred_lr)
print("Confusion Matrix:")
print(cm_lr)

# Classification Report 출력
report_lr = classification_report(y_test, y_pred_lr)
print("Classification Report:")
print(report_lr)


Confusion Matrix:
[[148   2]
 [  1   9]]
Classification Report:
              precision    recall  f1-score   support

       False       0.99      0.99      0.99       150
        True       0.82      0.90      0.86        10

    accuracy                           0.98       160
   macro avg       0.91      0.94      0.92       160
weighted avg       0.98      0.98      0.98       160



## 비교 분석 결과
---
- Logistic Regression의 장점:
정확도와 F1-Score 모두 매우 높은 성능을 보였습니다.
- True 클래스의 정밀도와 재현율이 상대적으로 좋습니다.
- False 클래스에 대해서도 우수한 성능을 보였으며, 전반적으로 매크로 평균과 가중 평균이 높았습니다.
- Decision Tree의 단점:
True 클래스에 대한 성능이 매우 떨어졌습니다. 정밀도와 재현율 모두 낮아서, True 클래스를 제대로 예측하지 못했습니다.
- False 클래스에서는 상대적으로 좋은 성능을 보였지만, True 클래스에서 문제가 발생하여 매크로 평균이 매우 낮아졌습니다.

### 회고
---

LNS가 없었더라면 오늘 안으로 끝마치지 못했을거 같네요. 데이터 확인과 전처리가 가장 중요하다는 것을 노드 공부를 하면서도 느꼈고, 직접 코드 작성하면서도 뼈져리게 느꼈습니다. 이건 노가다 같다는 느낌이 들정도로 데어터 전처리를 해야한다는 것을 다시 한번 느끼는 시간이었습니다. 그리고 중간에 시도했던 RandomForest 모델도 있긴했는데 생각보다 정확도나 여러 종합적인 결과가 좋지 않아서 알아봤더니 데이터 전처리에 문제가 있었을 수 있다고 하더군요..그냥 포기하고 다른 모델로 진행했습니다. 다행히 그전에 진행했던 모델보다는 나은 결과가 나온거 같습니다..
너무 어려워요 ai..